# Import packages

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

# 1. Load dataset

In [2]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data = train_data.fillna(method = 'bfill')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


# 2. Preprocessing

In [3]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)

In [4]:
SEQ_LEN = 5
(train_x, train_y), (val_x, val_y), (test_x, test_y) = make_train_val_test_data_single_turbine(train_data, SEQ_LEN)

100%|██████████| 134/134 [00:22<00:00,  5.84it/s]


Train data(X, y): (134, 192, 720, 11) (134, 192, 288)
Validation data(X, y): (134, 1, 720, 11) (134, 1, 288)
Test data(X, y): (134, 1, 720, 11) (134, 0)


# 3. Modeling

In [5]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-07-11 08:14:19.564872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 08:14:19.568382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 08:14:19.568556: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
tids = train_data['TurbID'].unique()

In [7]:
models = []
for turbID in tqdm(tids):
    model = Sequential([
        GRU(256, input_shape=train_x[turbID-1][0].shape),  # TurbID startswith 1, data, model startswith 0
        Dense(516, activation='relu'),
        Dense(288, activation='relu')
    ])
    optimizer = tf.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    models.append(model)

  0%|          | 0/134 [00:00<?, ?it/s]2022-07-11 08:14:19.750641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-11 08:14:19.751431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 08:14:19.751593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 08:14:19.751689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

# 4. Training

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

for model, tx, ty, vx, vy in tqdm(zip(models, train_x, train_y, val_x, val_y)):
    model.fit(tx, ty, validation_data=(vx, vy), epochs=1000, batch_size=32, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]);

0it [00:00, ?it/s]

Epoch 1/1000


2022-07-11 08:14:39.445955: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


4/6 [===================>..........] - ETA: 0s - loss: 504310.3125 - mae: 467.0744

2022-07-11 08:14:39.968685: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

132it [51:27, 22.33s/it]

Epoch 1/1000
6/6 [==============================] - 1s 79ms/step - loss: 291790.7188 - mae: 346.0527 - val_loss: 354824.6875 - val_mae: 388.6534
Epoch 2/1000
6/6 [==============================] - 0s 36ms/step - loss: 271972.6250 - mae: 335.3734 - val_loss: 323228.1875 - val_mae: 373.4122
Epoch 3/1000
6/6 [==============================] - 0s 33ms/step - loss: 244249.9375 - mae: 321.7864 - val_loss: 290612.8125 - val_mae: 358.3880
Epoch 4/1000
6/6 [==============================] - 0s 33ms/step - loss: 219714.9219 - mae: 314.0994 - val_loss: 263875.0625 - val_mae: 348.4486
Epoch 5/1000
6/6 [==============================] - 0s 37ms/step - loss: 201420.8750 - mae: 313.5459 - val_loss: 245533.1875 - val_mae: 346.4879
Epoch 6/1000
6/6 [==============================] - 0s 35ms/step - loss: 189862.6719 - mae: 316.8237 - val_loss: 232742.5156 - val_mae: 348.1338
Epoch 7/1000
6/6 [==============================] - 0s 34ms/step - loss: 183728.1250 - mae: 324.0093 - val_loss: 226657.0312 - val

133it [51:38, 19.08s/it]

Epoch 1/1000
6/6 [==============================] - 1s 79ms/step - loss: 312397.4062 - mae: 362.7587 - val_loss: 295964.8750 - val_mae: 397.2493
Epoch 2/1000
6/6 [==============================] - 0s 35ms/step - loss: 291284.6562 - mae: 350.5818 - val_loss: 265554.6875 - val_mae: 372.9929
Epoch 3/1000
6/6 [==============================] - 0s 37ms/step - loss: 262507.5625 - mae: 335.7625 - val_loss: 233035.6875 - val_mae: 347.8740
Epoch 4/1000
6/6 [==============================] - 0s 33ms/step - loss: 237110.3281 - mae: 327.0924 - val_loss: 206098.9688 - val_mae: 328.1302
Epoch 5/1000
6/6 [==============================] - 0s 31ms/step - loss: 217528.6250 - mae: 324.8134 - val_loss: 185393.8125 - val_mae: 317.2835
Epoch 6/1000
6/6 [==============================] - 0s 34ms/step - loss: 204287.5000 - mae: 327.0919 - val_loss: 170370.3438 - val_mae: 314.0130
Epoch 7/1000
6/6 [==============================] - 0s 34ms/step - loss: 196584.2500 - mae: 333.4998 - val_loss: 161211.4844 - val

134it [51:50, 23.21s/it]


# 5. Generate submission

In [9]:
for tid in tids:
    sample_submission.loc[sample_submission['TurbID'] == tid, 'Patv'] = models[tid-1].predict(test_x[tid-1]).reshape(-1)
sample_submission.to_csv(join(PATH.output, "proposed4.csv"), index=False)

1/1 [==============================] - 0s 215ms/step


1/1 [==============================] - 0s 211ms/step
